# Entry to Fall SCML Competition

In [1]:
# loading/normalizing data to start with
import numpy as np
import h5py

datafile = h5py.File('SCNeuronModelCompetition.mat')
movie = datafile.get('trainingmovie_mini') # training movie
movie = (movie - np.mean(movie)) / np.std(movie) # normalize
frhist = datafile.get('FRhist_tr') # firing rate histograms

### Model

The following simple model is the best I was able to find for this dataset. It achieves a loss of `.3423` vs the template model's `.3632`. I was able to sometimes get a slightly (<`.0004`) higher accuracy using one LSTM layer, but it didn't seem worth it due to the quadrupled parameter count per node. The second hidden layer decreased loss by only about `.0005`.

In [2]:
import keras
from keras.layers import TimeDistributed, Dense

model = keras.models.Sequential()

model.add(TimeDistributed(Dense(65, activation='sigmoid'), input_shape=movie.shape[1:]))
model.add(TimeDistributed(Dense(65, activation='sigmoid')))
model.add(TimeDistributed(Dense(frhist.shape[2], activation='softplus')))

model.summary()
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-7), loss='poisson')

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(movie, frhist, epochs=200, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 150, 65)           798785    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 150, 65)           4290      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 150, 54)           3564      
Total params: 806,639
Trainable params: 806,639
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/200
230/230 [==============================] - 4s 19ms/step - loss: 0.7635 - val_loss: 0.6873
Epoch 2/200
230/230 [==============================] - 4s 16ms/step - loss: 0.6523 - val_loss: 0.5970
Epoch 3/200
230/230 [==============================] - 4s 16ms/step - loss: 0.5690 - val_loss: 0.5253
Epoch 4/200
230/230 [============================

### Thoughts

There are a few potential explanations as to why more complicated structures do not improve accuracy. There might just not be enough data to make recurrent layers like LSTM useful, or there could truly be little time dependency in the dataset. Using conv/pool layers might destroy data useful for particular neurons as a result of the pooling operation. Given how shallow my best ANN model ended up being, other methods like SVM might be better suited to the problem.

Thanks Dr. Ito for an interesting competition!